In [ ]:
DB_URL = "Your_DB_URL"

In [13]:
import os
import time
import pandas as pd

from utils.log_config import get_logger
from utils.ingestion import ingest_db
from utils.db_config import get_engine

engine = get_engine(DB_URL)

DATA_DIR = "data"
SUPPORTED_EXTENSIONS = (".csv", ".xlsx", ".xls")

logger = get_logger("inventory_ingest", console=True)

logger.info("Connected to database")
print("🛢 CONNECTED SUCCESSFULLY")


def load_dataframe(file_path: str) -> pd.DataFrame:
    """Load CSV or Excel into DataFrame."""
    if file_path.endswith(".csv"):
        return pd.read_csv(file_path)
    elif file_path.endswith((".xlsx", ".xls")):
        return pd.read_excel(file_path)
    else:
        raise ValueError("Unsupported file format")


for file in os.listdir(DATA_DIR):

    if not file.lower().endswith(SUPPORTED_EXTENSIONS):
        continue

    start = time.time()
    file_path = os.path.join(DATA_DIR, file)

    # clean table name
    table_name = (
        os.path.splitext(file)[0]
        .lower()
        .replace(" ", "_")
        .replace("-", "_")
    )

    try:
        logger.info(f"Loading file: {file}")

        df = load_dataframe(file_path)

        logger.info(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")

        with engine.begin() as conn:
            ingest_db(df, table_name, conn)

        elapsed = (time.time() - start) / 60
        logger.info(f"Time taken for {file}: {elapsed:.2f} minutes")

    except Exception:
        logger.exception(f"Error processing file: {file}")


logger.info("All files ingested successfully")
print("🎉 ALL FILES DONE")

2026-02-05 11:37:48,910 | INFO | inventory_ingest | Connected to database
2026-02-05 11:37:48,912 | INFO | inventory_ingest | Loading file: blinkit_products.csv
2026-02-05 11:37:48,919 | INFO | inventory_ingest | Rows: 237, Columns: 10
2026-02-05 11:37:48,986 | INFO | inventory_ingest | Time taken for blinkit_products.csv: 0.00 minutes
2026-02-05 11:37:48,986 | INFO | inventory_ingest | Loading file: blinkit_marketing_performance.csv
2026-02-05 11:37:48,992 | INFO | inventory_ingest | Rows: 9864, Columns: 11


🛢 CONNECTED SUCCESSFULLY


2026-02-05 11:37:49,609 | INFO | inventory_ingest | Time taken for blinkit_marketing_performance.csv: 0.01 minutes
2026-02-05 11:37:49,609 | INFO | inventory_ingest | Loading file: blinkit_inventory.csv
2026-02-05 11:37:49,644 | INFO | inventory_ingest | Rows: 259752, Columns: 4
2026-02-05 11:37:54,992 | INFO | inventory_ingest | Time taken for blinkit_inventory.csv: 0.09 minutes
2026-02-05 11:37:54,993 | INFO | inventory_ingest | Loading file: blinkit_customer_feedback.csv
2026-02-05 11:37:55,030 | INFO | inventory_ingest | Rows: 50000, Columns: 8
2026-02-05 11:37:56,934 | INFO | inventory_ingest | Time taken for blinkit_customer_feedback.csv: 0.03 minutes
2026-02-05 11:37:56,934 | INFO | inventory_ingest | Loading file: blinkit_order_items.csv
2026-02-05 11:37:56,971 | INFO | inventory_ingest | Rows: 225238, Columns: 5
2026-02-05 11:38:02,836 | INFO | inventory_ingest | Time taken for blinkit_order_items.csv: 0.10 minutes
2026-02-05 11:38:02,837 | INFO | inventory_ingest | Loading fi

🎉 ALL FILES DONE
